<a href="https://colab.research.google.com/github/kushagra-93/Deep-Learning-topics/blob/main/keras_hyperparametertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('diabetes2.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [121]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim=8))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data = (X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 25ms/step - loss: 0.6726 - accuracy: 0.6205 - val_loss: 0.6376 - val_accuracy: 0.6364
Epoch 2/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6199 - accuracy: 0.6792 - val_loss: 0.5951 - val_accuracy: 0.7143
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5829 - accuracy: 0.7036 - val_loss: 0.5638 - val_accuracy: 0.7468
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5565 - accuracy: 0.7166 - val_loss: 0.5421 - val_accuracy: 0.7662
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5365 - accuracy: 0.7345 - val_loss: 0.5252 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5205 - accuracy: 0.7459 - val_loss: 0.5124 - val_accuracy: 0.7597
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5085 - accuracy: 0.7524 - val_loss: 0.5033 - val_accuracy: 0.7597
Epoch 8/100


## Select the best Optimizer

In [13]:
pip install -U keras-tuner

In [14]:
import keras_tuner as kt

In [15]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['Adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [16]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [21]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [22]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
optimizer: Adam
Score: 0.7662337422370911

Trial 1 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7467532753944397

Trial 0 summary
Hyperparameters:
optimizer: sgd
Score: 0.6623376607894897

Trial 2 summary
Hyperparameters:
optimizer: adadelta
Score: 0.6558441519737244


In [25]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5335 - accuracy: 0.7573 - val_loss: 0.5305 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5183 - accuracy: 0.7638 - val_loss: 0.5171 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5053 - accuracy: 0.7655 - val_loss: 0.5069 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4959 - accuracy: 0.7704 - val_loss: 0.4984 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4874 - accuracy: 0.7671 - val_loss: 0.4935 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4802 - accuracy: 0.7736 - val_loss: 0.4890 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4744 - accuracy: 0.7752 - val_loss: 0.4843 - val_accuracy: 0.7857
Epoch 14

## Selecting right number of neurons

In [53]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation= 'sigmoid'))

  model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [54]:
tuner =kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials=5,
                       directory= 'mydir',
                       project_name='number_neurons1')

In [55]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 11s


In [56]:
tuner.results_summary()

Results summary
Results in mydir/number_neurons1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 80
Score: 0.8051947951316833

Trial 2 summary
Hyperparameters:
units: 8
Score: 0.7857142686843872

Trial 1 summary
Hyperparameters:
units: 72
Score: 0.7792207598686218

Trial 4 summary
Hyperparameters:
units: 48
Score: 0.7727272510528564

Trial 0 summary
Hyperparameters:
units: 16
Score: 0.7272727489471436


In [57]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [59]:
model=tuner.get_best_models(num_models=1)[0]

In [61]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 34ms/step - loss: 0.3915 - accuracy: 0.8208 - val_loss: 0.4547 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3923 - accuracy: 0.8225 - val_loss: 0.4558 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3907 - accuracy: 0.8225 - val_loss: 0.4517 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3916 - accuracy: 0.8241 - val_loss: 0.4545 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3890 - accuracy: 0.8290 - val_loss: 0.4526 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3892 - accuracy: 0.8208 - val_loss: 0.4547 - val_accuracy: 0.8182
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3893 - accuracy: 0.8257 - val_loss: 0.4546 - val_accuracy: 0.8117
Epoch 14

## Selecting number of layers

In [71]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(80,activation='relu',input_dim=8))

  for i in range (hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(80,activation='relu'))

  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [72]:
tuner= kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='mydir',
                      project_name='number_layers1')

In [73]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [74]:
tuner.results_summary()

Results summary
Results in mydir/number_layers1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 3
Score: 0.798701286315918

Trial 2 summary
Hyperparameters:
num_layers: 7
Score: 0.798701286315918

Trial 1 summary
Hyperparameters:
num_layers: 9
Score: 0.7857142686843872


In [75]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [77]:
model = tuner.get_best_models(num_models=1)[0]

In [78]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4192 - accuracy: 0.7997 - val_loss: 0.4612 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4095 - accuracy: 0.7948 - val_loss: 0.4658 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4069 - accuracy: 0.7915 - val_loss: 0.4701 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3904 - accuracy: 0.8127 - val_loss: 0.4698 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3898 - accuracy: 0.8143 - val_loss: 0.4675 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3768 - accuracy: 0.8257 - val_loss: 0.4756 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3667 - accuracy: 0.8371 - val_loss: 0.4931 - val_accuracy: 0.8052
Epoch 14

## Selecting neurons, layers, optimizer together

In [126]:
def build_model(hp):
  model = Sequential()

  #counter = 0
  num_layers = hp.Int('num_layers', min_value=1, max_value=10)

  for i in range(num_layers):

    if i==0:
      units = hp.Int('units'+ str(i),min_value=8,max_value=128,step=8)
      activation = hp.Choice('activation' + str(i), values = ['relu','tanh','sigmoid'])
      dropout_rate = hp.Choice('dropout'+ str(i),values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
      model.add(Dense(units ,activation,input_dim=8))
      model.add(Dropout(dropout_rate))

    else:
      units = hp.Int('units'+ str(i),min_value=8,max_value=128,step=8)
      activation = hp.Choice('activation' + str(i), values = ['relu','tanh','sigmoid'])
      dropout_rate = hp.Choice('dropout'+ str(i),values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
      model.add(Dense(units ,activation))
      model.add(Dropout(dropout_rate))


  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values = ['Adam','sgd','rmsprop','nadam','adadelta']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [127]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'final')

In [128]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 14s


In [129]:
tuner.results_summary()

Results summary
Results in mydir/final
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 10
units0: 120
activation0: relu
dropout0: 0.7
optimizer: nadam
units1: 8
activation1: relu
dropout1: 0.1
units2: 8
activation2: relu
dropout2: 0.1
units3: 8
activation3: relu
dropout3: 0.1
units4: 8
activation4: relu
dropout4: 0.1
units5: 8
activation5: relu
dropout5: 0.1
units6: 8
activation6: relu
dropout6: 0.1
units7: 8
activation7: relu
dropout7: 0.1
units8: 8
activation8: relu
dropout8: 0.1
units9: 8
activation9: relu
dropout9: 0.1
Score: 0.6428571343421936

Trial 1 summary
Hyperparameters:
num_layers: 4
units0: 128
activation0: sigmoid
dropout0: 0.3
optimizer: adadelta
units1: 112
activation1: tanh
dropout1: 0.5
units2: 72
activation2: tanh
dropout2: 0.4
units3: 56
activation3: relu
dropout3: 0.8
units4: 80
activation4: sigmoid
dropout4: 0.8
units5: 128
activation5: sigmoid
dropout5: 0.3
units6: 48
activation6: sigmoid
dropou

In [130]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 120,
 'activation0': 'relu',
 'dropout0': 0.7,
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1,
 'units9': 8,
 'activation9': 'relu',
 'dropout9': 0.1}

In [131]:
model = tuner.get_best_models(num_models=1)[0]

In [132]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 [==============================] - 5s 19ms/step - loss: 0.6722 - accuracy: 0.6531 - val_loss: 0.6634 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6481 - accuracy: 0.6531 - val_loss: 0.6397 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6462 - accuracy: 0.6531 - val_loss: 0.6266 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6169 - accuracy: 0.6531 - val_loss: 0.6050 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6111 - accuracy: 0.6531 - val_loss: 0.5867 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5916 - accuracy: 0.6531 - val_loss: 0.5658 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5769 - accuracy: 0.6531 - val_loss: 0.5488 - val_accuracy: 0.6429
Epoch 14